# Energy Optimization Analysis

## Objectives:
1. Integrate all analysis results to identify optimization opportunities
2. Develop comprehensive energy management strategies
3. Quantify potential savings and efficiency improvements
4. Create actionable optimization recommendations
5. Design energy management control strategies

## Key Analyses:
- Multi-system optimization opportunities
- Cost-benefit analysis of optimization strategies
- Peak demand reduction potential
- Self-consumption optimization
- Load shifting and demand response strategies
- Energy storage optimization
- Integration of predictive control
- ROI analysis for optimization investments

In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import asyncio
import pickle
from scipy import optimize
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

# Add pems_v2 directory to path for imports
sys.path.append(str(Path('./pems_v2').resolve()))

# Import project modules
from analysis.core.data_extraction import DataExtractor
from config.settings import PEMSSettings

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 1. Load Previous Analysis Results

Load all previous analysis results to build comprehensive optimization strategies

In [ ]:
# Load previous analysis results
results_dir = Path('./data/processed')

# Initialize containers for analysis results
analysis_results = {}
analysis_summaries = {}

# Load heating pattern analysis
try:
    with open(results_dir / 'heating_pattern_analysis.pkl', 'rb') as f:
        analysis_results['heating_patterns'] = pickle.load(f)
    print("✓ Loaded heating pattern analysis")
except FileNotFoundError:
    print("⚠ Heating pattern analysis not found")
    analysis_results['heating_patterns'] = {}

# Load thermal analysis
try:
    with open(results_dir / 'thermal_analysis_results.pkl', 'rb') as f:
        analysis_results['thermal'] = pickle.load(f)
    print("✓ Loaded thermal analysis")
except FileNotFoundError:
    print("⚠ Thermal analysis not found")
    analysis_results['thermal'] = {}

# Load base load analysis
try:
    with open(results_dir / 'base_load_analysis_results.pkl', 'rb') as f:
        analysis_results['base_load'] = pickle.load(f)
    print("✓ Loaded base load analysis")
except FileNotFoundError:
    print("⚠ Base load analysis not found")
    analysis_results['base_load'] = {}

# Load weather correlation analysis
try:
    with open(results_dir / 'weather_correlation_results.pkl', 'rb') as f:
        analysis_results['weather_correlation'] = pickle.load(f)
    print("✓ Loaded weather correlation analysis")
except FileNotFoundError:
    print("⚠ Weather correlation analysis not found")
    analysis_results['weather_correlation'] = {}

# Load additional CSV data if available
try:
    thermal_params = pd.read_csv(results_dir / 'thermal_rc_parameters.csv', index_col=0)
    analysis_results['thermal_params_df'] = thermal_params
    print("✓ Loaded thermal RC parameters")
except FileNotFoundError:
    print("⚠ Thermal RC parameters not found")
    analysis_results['thermal_params_df'] = pd.DataFrame()

try:
    weather_correlations = pd.read_csv(results_dir / 'weather_correlations.csv')
    analysis_results['weather_correlations_df'] = weather_correlations
    print("✓ Loaded weather correlations")
except FileNotFoundError:
    print("⚠ Weather correlations not found")
    analysis_results['weather_correlations_df'] = pd.DataFrame()

print(f"\nLoaded {len(analysis_results)} analysis components")
print(f"Available analyses: {list(analysis_results.keys())}")

## 2. Extract Key Metrics from Previous Analyses

Extract and consolidate key findings from all analyses

In [ ]:
# Extract key metrics for optimization analysis
optimization_metrics = {}

# Extract heating pattern metrics
heating_analysis = analysis_results.get('heating_patterns', {})
if heating_analysis:
    heating_metrics = {
        'rooms_analyzed': len(heating_analysis.get('rooms_analyzed', [])),
        'average_utilization': 0,
        'peak_demand_potential': 0,
        'switching_efficiency': 0
    }
    
    # Calculate average room utilization
    daily_patterns = heating_analysis.get('daily_patterns', {})
    if daily_patterns:
        total_duty_cycles = []
        for room, patterns in daily_patterns.items():
            if 'all_days' in patterns and not patterns['all_days'].empty:
                avg_duty = patterns['all_days'].mean()
                total_duty_cycles.append(avg_duty)
        
        if total_duty_cycles:
            heating_metrics['average_utilization'] = np.mean(total_duty_cycles)
    
    # Extract switching analysis
    switching_analysis = heating_analysis.get('switching_analysis', {})
    if switching_analysis:
        switches_per_day = [stats.get('switches_per_day', 0) for stats in switching_analysis.values()]
        if switches_per_day:
            heating_metrics['avg_switches_per_day'] = np.mean(switches_per_day)
            heating_metrics['high_switching_rooms'] = sum(1 for s in switches_per_day if s > 20)
    
    optimization_metrics['heating'] = heating_metrics
    print(f"Heating metrics: {heating_metrics}")

# Extract thermal analysis metrics
thermal_analysis = analysis_results.get('thermal', {})
if thermal_analysis:
    thermal_metrics = {
        'rooms_with_models': len(thermal_analysis.get('rooms_analyzed', [])),
        'avg_time_constant': 0,
        'thermal_efficiency': 0,
        'model_accuracy': 0
    }
    
    # Extract thermal parameters
    thermal_params = thermal_analysis.get('thermal_parameters', {})
    if thermal_params:
        time_constants = []
        for room, params in thermal_params.items():
            if 'heating_tau_avg' in params:
                time_constants.append(params['heating_tau_avg'])
        
        if time_constants:
            thermal_metrics['avg_time_constant'] = np.mean(time_constants)
            thermal_metrics['fastest_room'] = min(time_constants)
            thermal_metrics['slowest_room'] = max(time_constants)
    
    # Extract RC parameters
    rc_params = thermal_analysis.get('rc_parameters', {})
    if rc_params:
        thermal_resistances = [params.get('R_thermal', 0) for params in rc_params.values()]
        if thermal_resistances:
            thermal_metrics['avg_thermal_resistance'] = np.mean(thermal_resistances)
            thermal_metrics['total_thermal_mass'] = sum([params.get('thermal_mass_kj_k', 0) for params in rc_params.values()])
    
    optimization_metrics['thermal'] = thermal_metrics
    print(f"Thermal metrics: {thermal_metrics}")

# Extract base load metrics
base_load_analysis = analysis_results.get('base_load', {})
if base_load_analysis:
    base_load_metrics = {
        'analysis_period_days': 0,
        'load_factor': 0,
        'base_load_ratio': 0,
        'forecast_accuracy': 0
    }
    
    consumption_summary = base_load_analysis.get('consumption_summary', {})
    if consumption_summary:
        base_load_metrics['analysis_period_days'] = consumption_summary.get('total_hours', 0) / 24
        
        efficiency_metrics = consumption_summary.get('efficiency_metrics', {})
        if efficiency_metrics:
            base_load_metrics['load_factor'] = efficiency_metrics.get('load_factor', 0)
            base_load_metrics['peak_power_w'] = efficiency_metrics.get('peak_power_w', 0)
            base_load_metrics['avg_power_w'] = efficiency_metrics.get('avg_power_w', 0)
            base_load_metrics['daily_energy_kwh'] = efficiency_metrics.get('daily_energy_mean', 0)
    
    # Extract model performance
    model_results = base_load_analysis.get('model_results', {})
    if model_results:
        best_model_r2 = max([results.get('r2_mean', 0) for results in model_results.values()], default=0)
        base_load_metrics['forecast_accuracy'] = best_model_r2
    
    optimization_metrics['base_load'] = base_load_metrics
    print(f"Base load metrics: {base_load_metrics}")

# Extract weather correlation metrics
weather_analysis = analysis_results.get('weather_correlation', {})
if weather_analysis:
    weather_metrics = {
        'pv_weather_correlation': 0,
        'heating_weather_correlation': 0,
        'prediction_models_available': 0,
        'extreme_events_identified': 0
    }
    
    # PV-weather correlations
    pv_weather = weather_analysis.get('pv_weather_analysis', {})
    if pv_weather and 'correlations' in pv_weather:
        strongest_pv_corr = max([abs(corr) for corr in pv_weather['correlations'].values()], default=0)
        weather_metrics['pv_weather_correlation'] = strongest_pv_corr
    
    # Heating-weather correlations
    heating_weather = weather_analysis.get('heating_weather_analysis', {})
    if heating_weather and 'correlations' in heating_weather:
        strongest_heating_corr = max([abs(corr) for corr in heating_weather['correlations'].values()], default=0)
        weather_metrics['heating_weather_correlation'] = strongest_heating_corr
    
    # Prediction models
    prediction_models = weather_analysis.get('weather_prediction_models', {})
    weather_metrics['prediction_models_available'] = len(prediction_models)
    
    # Extreme weather events
    extreme_weather = weather_analysis.get('extreme_weather_analysis', {})
    if extreme_weather:
        weather_metrics['extreme_events_identified'] = len(extreme_weather.get('events', {}))
    
    optimization_metrics['weather'] = weather_metrics
    print(f"Weather metrics: {weather_metrics}")

print(f"\nExtracted optimization metrics for {len(optimization_metrics)} systems")

## 3. System Integration Analysis

Analyze interactions between different energy systems

In [ ]:
# Analyze system interactions and integration opportunities
system_integration = {}

# Calculate heating demand vs PV production correlation
if 'heating' in optimization_metrics and 'base_load' in optimization_metrics:
    
    # Heating system sizing analysis
    heating_metrics = optimization_metrics['heating']
    base_load_metrics = optimization_metrics['base_load']
    
    if heating_metrics.get('rooms_analyzed', 0) > 0:
        # Calculate heating system characteristics
        total_heating_capacity = heating_metrics.get('rooms_analyzed', 0) * 1500  # Assume 1.5kW average per room
        avg_consumption = base_load_metrics.get('avg_power_w', 0)
        peak_consumption = base_load_metrics.get('peak_power_w', 0)
        
        heating_integration = {
            'total_heating_capacity_w': total_heating_capacity,
            'heating_to_consumption_ratio': total_heating_capacity / avg_consumption if avg_consumption > 0 else 0,
            'heating_peak_impact': total_heating_capacity / peak_consumption if peak_consumption > 0 else 0,
            'utilization_rate': heating_metrics.get('average_utilization', 0)
        }
        
        system_integration['heating_consumption'] = heating_integration
        
        print(f"Heating-Consumption Integration:")
        print(f"  Total heating capacity: {total_heating_capacity/1000:.1f} kW")
        print(f"  Ratio to avg consumption: {heating_integration['heating_to_consumption_ratio']:.2f}")
        print(f"  Current utilization: {heating_integration['utilization_rate']*100:.1f}%")

# PV-Consumption integration analysis
if 'weather' in optimization_metrics and 'base_load' in optimization_metrics:
    weather_metrics = optimization_metrics['weather']
    
    # Estimate PV system characteristics
    pv_correlation = weather_metrics.get('pv_weather_correlation', 0)
    
    if pv_correlation > 0.5:  # Good weather correlation indicates active PV system
        # Estimate self-consumption potential
        daily_energy = base_load_metrics.get('daily_energy_kwh', 0)
        load_factor = base_load_metrics.get('load_factor', 0)
        
        pv_integration = {
            'weather_predictability': pv_correlation,
            'daily_consumption_kwh': daily_energy,
            'load_stability': load_factor,
            'self_consumption_potential': min(1.0, load_factor * 1.2),  # Heuristic estimate
            'storage_benefit_score': (1 - load_factor) * pv_correlation  # Higher for variable loads with good weather prediction
        }
        
        system_integration['pv_consumption'] = pv_integration
        
        print(f"\nPV-Consumption Integration:")
        print(f"  Weather predictability: {pv_correlation:.2f}")
        print(f"  Self-consumption potential: {pv_integration['self_consumption_potential']*100:.1f}%")
        print(f"  Storage benefit score: {pv_integration['storage_benefit_score']:.2f}")

# Thermal-Weather integration
if 'thermal' in optimization_metrics and 'weather' in optimization_metrics:
    thermal_metrics = optimization_metrics['thermal']
    weather_metrics = optimization_metrics['weather']
    
    heating_weather_corr = weather_metrics.get('heating_weather_correlation', 0)
    avg_time_constant = thermal_metrics.get('avg_time_constant', 0)
    
    thermal_weather_integration = {
        'weather_responsiveness': heating_weather_corr,
        'thermal_inertia_hours': avg_time_constant,
        'preheating_potential': avg_time_constant / 24,  # Fraction of day for preheating
        'weather_prediction_value': heating_weather_corr * (avg_time_constant / 6)  # Value of 6+ hour weather prediction
    }
    
    system_integration['thermal_weather'] = thermal_weather_integration
    
    print(f"\nThermal-Weather Integration:")
    print(f"  Weather responsiveness: {heating_weather_corr:.2f}")
    print(f"  Average thermal inertia: {avg_time_constant:.1f} hours")
    print(f"  Preheating potential: {thermal_weather_integration['preheating_potential']*100:.1f}%")

# Overall system synergy score
synergy_scores = []
for integration_type, metrics in system_integration.items():
    if integration_type == 'heating_consumption':
        # Higher score for good utilization and manageable heating load
        utilization = metrics.get('utilization_rate', 0)
        ratio = metrics.get('heating_to_consumption_ratio', 0)
        score = utilization * (1 / (1 + ratio))  # Penalize oversized heating
        synergy_scores.append(score)
    
    elif integration_type == 'pv_consumption':
        # Higher score for good predictability and storage benefit
        predictability = metrics.get('weather_predictability', 0)
        storage_benefit = metrics.get('storage_benefit_score', 0)
        score = (predictability + storage_benefit) / 2
        synergy_scores.append(score)
    
    elif integration_type == 'thermal_weather':
        # Higher score for good weather response and useful thermal inertia
        responsiveness = metrics.get('weather_responsiveness', 0)
        inertia_value = min(1.0, metrics.get('thermal_inertia_hours', 0) / 12)  # Optimal around 12 hours
        score = responsiveness * inertia_value
        synergy_scores.append(score)

if synergy_scores:
    overall_synergy = np.mean(synergy_scores)
    system_integration['overall_synergy_score'] = overall_synergy
    
    print(f"\nOverall System Synergy Score: {overall_synergy:.2f}")
    
    if overall_synergy > 0.6:
        synergy_rating = "Excellent - High integration potential"
    elif overall_synergy > 0.4:
        synergy_rating = "Good - Moderate integration opportunities"
    elif overall_synergy > 0.2:
        synergy_rating = "Fair - Some integration possible"
    else:
        synergy_rating = "Poor - Limited integration benefits"
    
    print(f"Synergy Rating: {synergy_rating}")
else:
    print("\nInsufficient data for system integration analysis")

## 4. Optimization Opportunity Identification

Identify and prioritize specific optimization opportunities

In [ ]:
# Identify and prioritize optimization opportunities
optimization_opportunities = []

# Heating system optimization opportunities
if 'heating' in optimization_metrics:
    heating_metrics = optimization_metrics['heating']
    
    # High switching frequency optimization
    high_switching_rooms = heating_metrics.get('high_switching_rooms', 0)
    if high_switching_rooms > 0:
        opportunity = {
            'category': 'Heating Control',
            'opportunity': 'Reduce Relay Switching Frequency',
            'description': f'Implement hysteresis control for {high_switching_rooms} rooms with high switching',
            'impact': 'Medium',
            'effort': 'Low',
            'priority_score': 7,
            'potential_savings': '5-10% heating system wear reduction',
            'implementation': 'Adjust thermostat deadband and control logic'
        }
        optimization_opportunities.append(opportunity)
    
    # Heating schedule optimization
    avg_utilization = heating_metrics.get('average_utilization', 0)
    if avg_utilization > 0.5:
        opportunity = {
            'category': 'Heating Schedule',
            'opportunity': 'Optimize Heating Schedules',
            'description': f'Current utilization {avg_utilization*100:.1f}% indicates potential for schedule optimization',
            'impact': 'High',
            'effort': 'Medium',
            'priority_score': 8,
            'potential_savings': '10-20% heating energy reduction',
            'implementation': 'Implement occupancy-based and weather-predictive scheduling'
        }
        optimization_opportunities.append(opportunity)

# Thermal system optimization
if 'thermal' in optimization_metrics:
    thermal_metrics = optimization_metrics['thermal']
    
    # Preheating optimization
    avg_time_constant = thermal_metrics.get('avg_time_constant', 0)
    if avg_time_constant > 3:  # Rooms with significant thermal inertia
        opportunity = {
            'category': 'Thermal Optimization',
            'opportunity': 'Implement Predictive Preheating',
            'description': f'Rooms have {avg_time_constant:.1f}h thermal inertia - suitable for preheating strategies',
            'impact': 'High',
            'effort': 'High',
            'priority_score': 9,
            'potential_savings': '15-25% heating cost reduction through price optimization',
            'implementation': 'Develop thermal model-based predictive control with weather forecasting'
        }
        optimization_opportunities.append(opportunity)
    
    # Thermal efficiency improvements
    avg_resistance = thermal_metrics.get('avg_thermal_resistance', 0)
    if avg_resistance > 0.002:  # High thermal resistance indicates poor insulation
        opportunity = {
            'category': 'Building Efficiency',
            'opportunity': 'Improve Thermal Insulation',
            'description': f'High thermal resistance ({avg_resistance:.4f} K/W) indicates insulation improvement potential',
            'impact': 'High',
            'effort': 'High',
            'priority_score': 6,
            'potential_savings': '20-30% heating demand reduction',
            'implementation': 'Physical insulation upgrades to building envelope'
        }
        optimization_opportunities.append(opportunity)

# Base load optimization
if 'base_load' in optimization_metrics:
    base_load_metrics = optimization_metrics['base_load']
    
    # Load factor improvement
    load_factor = base_load_metrics.get('load_factor', 0)
    if load_factor < 0.6:
        opportunity = {
            'category': 'Load Management',
            'opportunity': 'Improve Load Factor',
            'description': f'Current load factor {load_factor:.2f} indicates high peak/average ratio',
            'impact': 'Medium',
            'effort': 'Medium',
            'priority_score': 7,
            'potential_savings': '10-15% demand charge reduction',
            'implementation': 'Implement load shifting and peak shaving strategies'
        }
        optimization_opportunities.append(opportunity)
    
    # Demand response potential
    forecast_accuracy = base_load_metrics.get('forecast_accuracy', 0)
    if forecast_accuracy > 0.7:
        opportunity = {
            'category': 'Demand Response',
            'opportunity': 'Implement Predictive Load Management',
            'description': f'Good forecast accuracy ({forecast_accuracy:.2f}) enables predictive optimization',
            'impact': 'High',
            'effort': 'Medium',
            'priority_score': 8,
            'potential_savings': '5-15% energy cost reduction through time-of-use optimization',
            'implementation': 'Deploy ML-based load forecasting with automated demand response'
        }
        optimization_opportunities.append(opportunity)

# Weather-based optimization
if 'weather' in optimization_metrics:
    weather_metrics = optimization_metrics['weather']
    
    # PV production optimization
    pv_correlation = weather_metrics.get('pv_weather_correlation', 0)
    if pv_correlation > 0.7:
        opportunity = {
            'category': 'PV Optimization',
            'opportunity': 'Weather-Based PV Production Forecasting',
            'description': f'Strong weather correlation ({pv_correlation:.2f}) enables accurate PV forecasting',
            'impact': 'High',
            'effort': 'Medium',
            'priority_score': 9,
            'potential_savings': '10-20% improvement in self-consumption through better forecasting',
            'implementation': 'Implement weather-based PV production models with battery optimization'
        }
        optimization_opportunities.append(opportunity)
    
    # Heating demand prediction
    heating_correlation = weather_metrics.get('heating_weather_correlation', 0)
    if heating_correlation > 0.6:
        opportunity = {
            'category': 'Weather Integration',
            'opportunity': 'Weather-Adaptive Heating Control',
            'description': f'Strong heating-weather correlation ({heating_correlation:.2f}) enables weather-adaptive control',
            'impact': 'Medium',
            'effort': 'Medium',
            'priority_score': 7,
            'potential_savings': '8-12% heating energy reduction through weather-adaptive control',
            'implementation': 'Integrate weather forecasting into heating control algorithms'
        }
        optimization_opportunities.append(opportunity)

# System integration opportunities
if system_integration:
    overall_synergy = system_integration.get('overall_synergy_score', 0)
    
    if overall_synergy > 0.5:
        opportunity = {
            'category': 'System Integration',
            'opportunity': 'Implement Holistic Energy Management',
            'description': f'High system synergy ({overall_synergy:.2f}) enables integrated optimization',
            'impact': 'Very High',
            'effort': 'High',
            'priority_score': 10,
            'potential_savings': '20-35% total energy cost reduction through integrated optimization',
            'implementation': 'Deploy comprehensive energy management system with predictive multi-system optimization'
        }
        optimization_opportunities.append(opportunity)
    
    # Storage optimization
    pv_integration = system_integration.get('pv_consumption', {})
    storage_benefit = pv_integration.get('storage_benefit_score', 0)
    if storage_benefit > 0.3:
        opportunity = {
            'category': 'Energy Storage',
            'opportunity': 'Optimize Battery Storage Strategy',
            'description': f'Storage benefit score {storage_benefit:.2f} indicates significant storage optimization potential',
            'impact': 'High',
            'effort': 'Medium',
            'priority_score': 8,
            'potential_savings': '15-25% reduction in grid energy costs through optimized storage',
            'implementation': 'Implement dynamic battery management with price and weather forecasting'
        }
        optimization_opportunities.append(opportunity)

# Sort opportunities by priority score
optimization_opportunities.sort(key=lambda x: x['priority_score'], reverse=True)

print(f"\nIdentified {len(optimization_opportunities)} Optimization Opportunities:")
print("=" * 80)

for i, opp in enumerate(optimization_opportunities, 1):
    print(f"{i}. {opp['opportunity']} (Priority: {opp['priority_score']}/10)")
    print(f"   Category: {opp['category']}")
    print(f"   Impact: {opp['impact']}, Effort: {opp['effort']}")
    print(f"   Savings: {opp['potential_savings']}")
    print(f"   Description: {opp['description']}")
    print()

## 5. Cost-Benefit Analysis

Analyze costs and benefits of optimization strategies

In [ ]:
# Cost-benefit analysis for optimization opportunities

# Define baseline energy consumption and costs
baseline_metrics = {
    'daily_energy_kwh': optimization_metrics.get('base_load', {}).get('daily_energy_kwh', 50),
    'energy_price_czk_kwh': 6.0,  # Average Czech electricity price
    'annual_energy_cost_czk': 0,
    'peak_demand_kw': optimization_metrics.get('base_load', {}).get('peak_power_w', 10000) / 1000,
    'demand_charge_czk_kw_month': 150  # Typical demand charge
}

# Calculate annual baseline costs
baseline_metrics['annual_energy_cost_czk'] = (
    baseline_metrics['daily_energy_kwh'] * 365 * baseline_metrics['energy_price_czk_kwh']
)
baseline_metrics['annual_demand_cost_czk'] = (
    baseline_metrics['peak_demand_kw'] * baseline_metrics['demand_charge_czk_kw_month'] * 12
)
baseline_metrics['total_annual_cost_czk'] = (
    baseline_metrics['annual_energy_cost_czk'] + baseline_metrics['annual_demand_cost_czk']
)

print(f"Baseline Energy Economics:")
print(f"  Daily consumption: {baseline_metrics['daily_energy_kwh']:.1f} kWh")
print(f"  Peak demand: {baseline_metrics['peak_demand_kw']:.1f} kW")
print(f"  Annual energy cost: {baseline_metrics['annual_energy_cost_czk']:,.0f} CZK")
print(f"  Annual demand cost: {baseline_metrics['annual_demand_cost_czk']:,.0f} CZK")
print(f"  Total annual cost: {baseline_metrics['total_annual_cost_czk']:,.0f} CZK")

# Cost-benefit analysis for each opportunity
cost_benefit_analysis = []

for opp in optimization_opportunities:
    # Extract savings percentage from opportunity description
    savings_text = opp['potential_savings']
    
    # Parse savings percentage (extract first number)
    import re
    savings_match = re.search(r'(\d+)(?:-\d+)?%', savings_text)
    if savings_match:
        savings_pct = float(savings_match.group(1)) / 100
    else:
        savings_pct = 0.1  # Default 10% if no percentage found
    
    # Estimate implementation costs based on effort and category
    implementation_costs = {
        'Low': 10000,    # 10k CZK for software/control changes
        'Medium': 50000, # 50k CZK for system upgrades
        'High': 150000   # 150k CZK for major installations
    }
    
    effort = opp['effort']
    base_cost = implementation_costs.get(effort, 50000)
    
    # Adjust cost based on category
    category_multipliers = {
        'Heating Control': 0.5,
        'Heating Schedule': 0.3,
        'Thermal Optimization': 1.5,
        'Building Efficiency': 3.0,
        'Load Management': 0.8,
        'Demand Response': 1.0,
        'PV Optimization': 1.2,
        'Weather Integration': 1.0,
        'System Integration': 2.0,
        'Energy Storage': 1.5
    }
    
    cost_multiplier = category_multipliers.get(opp['category'], 1.0)
    implementation_cost = base_cost * cost_multiplier
    
    # Calculate annual savings
    if 'heating' in savings_text.lower() or 'thermal' in savings_text.lower():
        # Heating-related savings (assume 60% of total energy is heating)
        affected_cost = baseline_metrics['total_annual_cost_czk'] * 0.6
    elif 'demand' in savings_text.lower() or 'peak' in savings_text.lower():
        # Demand charge savings
        affected_cost = baseline_metrics['annual_demand_cost_czk']
    elif 'cost' in savings_text.lower():
        # Total cost savings
        affected_cost = baseline_metrics['total_annual_cost_czk']
    else:
        # Energy savings
        affected_cost = baseline_metrics['annual_energy_cost_czk']
    
    annual_savings = affected_cost * savings_pct
    
    # Calculate payback period and ROI
    payback_years = implementation_cost / annual_savings if annual_savings > 0 else float('inf')
    roi_5_year = (annual_savings * 5 - implementation_cost) / implementation_cost if implementation_cost > 0 else 0
    
    cost_benefit = {
        'opportunity': opp['opportunity'],
        'category': opp['category'],
        'priority_score': opp['priority_score'],
        'implementation_cost_czk': implementation_cost,
        'annual_savings_czk': annual_savings,
        'payback_years': payback_years,
        'roi_5_year': roi_5_year,
        'npv_5_year_czk': annual_savings * 5 - implementation_cost,
        'savings_percentage': savings_pct * 100
    }
    
    cost_benefit_analysis.append(cost_benefit)

# Sort by ROI
cost_benefit_analysis.sort(key=lambda x: x['roi_5_year'], reverse=True)

print(f"\n\nCost-Benefit Analysis:")
print("=" * 100)
print(f"{'Opportunity':30s} {'Cost (CZK)':12s} {'Annual Savings':15s} {'Payback':10s} {'5yr ROI':10s} {'NPV 5yr':12s}")
print("-" * 100)

for cb in cost_benefit_analysis:
    payback_str = f"{cb['payback_years']:.1f}y" if cb['payback_years'] < 10 else ">10y"
    roi_str = f"{cb['roi_5_year']*100:+.0f}%"
    npv_str = f"{cb['npv_5_year_czk']:,.0f}"
    
    print(f"{cb['opportunity'][:29]:30s} {cb['implementation_cost_czk']:11,.0f} "
          f"{cb['annual_savings_czk']:14,.0f} {payback_str:9s} {roi_str:9s} {npv_str:11s}")

# Calculate portfolio metrics
total_investment = sum([cb['implementation_cost_czk'] for cb in cost_benefit_analysis])
total_annual_savings = sum([cb['annual_savings_czk'] for cb in cost_benefit_analysis])
portfolio_payback = total_investment / total_annual_savings if total_annual_savings > 0 else float('inf')
portfolio_roi = (total_annual_savings * 5 - total_investment) / total_investment if total_investment > 0 else 0

print("-" * 100)
print(f"{'PORTFOLIO TOTAL':30s} {total_investment:11,.0f} {total_annual_savings:14,.0f} "
      f"{portfolio_payback:.1f}y    {portfolio_roi*100:+.0f}%    {total_annual_savings * 5 - total_investment:11,.0f}")

print(f"\nPortfolio Summary:")
print(f"  Total investment required: {total_investment:,.0f} CZK")
print(f"  Total annual savings: {total_annual_savings:,.0f} CZK")
print(f"  Portfolio payback period: {portfolio_payback:.1f} years")
print(f"  5-year total savings: {total_annual_savings * 5:,.0f} CZK")
print(f"  5-year net profit: {total_annual_savings * 5 - total_investment:,.0f} CZK")
print(f"  Return on investment (5yr): {portfolio_roi*100:+.1f}%")

# Identify quick wins (payback < 2 years)
quick_wins = [cb for cb in cost_benefit_analysis if cb['payback_years'] < 2]
if quick_wins:
    print(f"\nQuick Wins (Payback < 2 years):")
    for qw in quick_wins:
        print(f"  - {qw['opportunity']}: {qw['payback_years']:.1f}y payback, {qw['annual_savings_czk']:,.0f} CZK/year")

# Store cost-benefit results
optimization_economics = {
    'baseline_metrics': baseline_metrics,
    'cost_benefit_analysis': cost_benefit_analysis,
    'portfolio_metrics': {
        'total_investment_czk': total_investment,
        'total_annual_savings_czk': total_annual_savings,
        'payback_years': portfolio_payback,
        'roi_5_year': portfolio_roi,
        'npv_5_year_czk': total_annual_savings * 5 - total_investment
    },
    'quick_wins': len(quick_wins)
}

## 6. Implementation Roadmap

Create a phased implementation plan for optimization strategies

In [ ]:
# Create implementation roadmap

# Define implementation phases
phases = {
    'Phase 1 - Quick Wins (0-6 months)': {
        'criteria': lambda cb: cb['payback_years'] < 2 and cb['implementation_cost_czk'] < 50000,
        'focus': 'Low-cost, high-impact improvements',
        'opportunities': []
    },
    'Phase 2 - Medium-term (6-18 months)': {
        'criteria': lambda cb: 2 <= cb['payback_years'] < 4 and cb['roi_5_year'] > 0.5,
        'focus': 'System integration and predictive control',
        'opportunities': []
    },
    'Phase 3 - Long-term (18+ months)': {
        'criteria': lambda cb: cb['payback_years'] >= 4 or cb['implementation_cost_czk'] >= 100000,
        'focus': 'Infrastructure upgrades and advanced optimization',
        'opportunities': []
    }
}

# Assign opportunities to phases
for cb in cost_benefit_analysis:
    assigned = False
    for phase_name, phase_info in phases.items():
        if phase_info['criteria'](cb) and not assigned:
            phase_info['opportunities'].append(cb)
            assigned = True
            break
    
    # If not assigned to any phase, put in Phase 3
    if not assigned:
        phases['Phase 3 - Long-term (18+ months)']['opportunities'].append(cb)

# Display implementation roadmap
print("\nImplementation Roadmap:")
print("=" * 80)

total_cumulative_investment = 0
total_cumulative_savings = 0

for phase_name, phase_info in phases.items():
    opportunities = phase_info['opportunities']
    
    if opportunities:
        phase_investment = sum([opp['implementation_cost_czk'] for opp in opportunities])
        phase_annual_savings = sum([opp['annual_savings_czk'] for opp in opportunities])
        
        total_cumulative_investment += phase_investment
        total_cumulative_savings += phase_annual_savings
        
        print(f"\n{phase_name}")
        print(f"Focus: {phase_info['focus']}")
        print(f"Investment: {phase_investment:,.0f} CZK")
        print(f"Annual savings: {phase_annual_savings:,.0f} CZK")
        print(f"Cumulative investment: {total_cumulative_investment:,.0f} CZK")
        print(f"Cumulative annual savings: {total_cumulative_savings:,.0f} CZK")
        
        print(f"\nOpportunities in this phase:")
        for opp in opportunities:
            print(f"  • {opp['opportunity']}")
            print(f"    Cost: {opp['implementation_cost_czk']:,.0f} CZK, "
                  f"Savings: {opp['annual_savings_czk']:,.0f} CZK/year, "
                  f"Payback: {opp['payback_years']:.1f}y")

# Create detailed implementation plan for Phase 1
phase1_opportunities = phases['Phase 1 - Quick Wins (0-6 months)']['opportunities']

if phase1_opportunities:
    print(f"\n\nDetailed Phase 1 Implementation Plan:")
    print("=" * 60)
    
    # Sort Phase 1 by payback period
    phase1_opportunities.sort(key=lambda x: x['payback_years'])
    
    implementation_timeline = []
    
    for i, opp in enumerate(phase1_opportunities):
        start_month = i * 2  # Stagger implementations every 2 months
        end_month = start_month + 3  # 3-month implementation period
        
        timeline_item = {
            'opportunity': opp['opportunity'],
            'start_month': start_month,
            'end_month': end_month,
            'cost': opp['implementation_cost_czk'],
            'priority': i + 1,
            'dependencies': [],
            'key_activities': []
        }
        
        # Define key activities based on category
        category = opp['category']
        if category == 'Heating Control':
            timeline_item['key_activities'] = [
                'Analyze current thermostat settings',
                'Implement hysteresis control logic',
                'Test and validate switching reduction',
                'Monitor system performance'
            ]
        elif category == 'Heating Schedule':
            timeline_item['key_activities'] = [
                'Analyze current heating patterns',
                'Develop occupancy-based schedules',
                'Implement schedule optimization',
                'Validate energy savings'
            ]
        elif category == 'Load Management':
            timeline_item['key_activities'] = [
                'Install load monitoring systems',
                'Develop load shifting algorithms',
                'Implement automated load control',
                'Optimize peak demand management'
            ]
        elif category == 'Demand Response':
            timeline_item['key_activities'] = [
                'Deploy forecasting models',
                'Integrate with energy markets',
                'Implement automated response',
                'Validate cost savings'
            ]
        else:
            timeline_item['key_activities'] = [
                'System analysis and design',
                'Implementation and testing',
                'Performance validation',
                'Monitoring and optimization'
            ]
        
        implementation_timeline.append(timeline_item)
    
    print(f"Phase 1 Timeline ({len(implementation_timeline)} projects):")
    print(f"{'Project':30s} {'Months':10s} {'Cost (CZK)':12s} {'Key Activities':30s}")
    print("-" * 85)
    
    for item in implementation_timeline:
        activities_str = ', '.join(item['key_activities'][:2]) + '...' if len(item['key_activities']) > 2 else ', '.join(item['key_activities'])
        print(f"{item['opportunity'][:29]:30s} {item['start_month']:2d}-{item['end_month']:2d} "
              f"{item['cost']:11,.0f} {activities_str[:29]:30s}")
    
    # Calculate Phase 1 milestones
    phase1_investment = sum([item['cost'] for item in implementation_timeline])
    phase1_savings = sum([opp['annual_savings_czk'] for opp in phase1_opportunities])
    
    print(f"\nPhase 1 Success Metrics:")
    print(f"  Total investment: {phase1_investment:,.0f} CZK")
    print(f"  Expected annual savings: {phase1_savings:,.0f} CZK")
    print(f"  6-month savings target: {phase1_savings/2:,.0f} CZK")
    print(f"  ROI after 1 year: {(phase1_savings - phase1_investment)/phase1_investment*100:+.1f}%")

# Store implementation roadmap
implementation_roadmap = {
    'phases': phases,
    'phase1_timeline': implementation_timeline if 'implementation_timeline' in locals() else [],
    'cumulative_metrics': {
        'total_investment': total_cumulative_investment,
        'total_annual_savings': total_cumulative_savings,
        'implementation_period_months': 24
    }
}

## 7. Risk Analysis and Mitigation

Analyze risks and develop mitigation strategies for optimization implementation

In [ ]:
# Risk analysis for optimization implementation

risk_assessment = {
    'technical_risks': [
        {
            'risk': 'Model Accuracy Degradation',
            'description': 'Weather and load prediction models may lose accuracy over time',
            'probability': 'Medium',
            'impact': 'Medium',
            'mitigation': [
                'Implement model performance monitoring',
                'Set up automated model retraining',
                'Maintain backup control strategies',
                'Regular model validation against actual data'
            ]
        },
        {
            'risk': 'System Integration Failures',
            'description': 'Complex system interactions may cause unexpected behavior',
            'probability': 'Low',
            'impact': 'High',
            'mitigation': [
                'Implement gradual rollout with testing phases',
                'Maintain manual override capabilities',
                'Comprehensive system testing before deployment',
                'Real-time monitoring and alerting systems'
            ]
        },
        {
            'risk': 'Communication System Failures',
            'description': 'MQTT or network failures could disrupt optimization',
            'probability': 'Medium',
            'impact': 'Medium',
            'mitigation': [
                'Implement redundant communication paths',
                'Local fallback control systems',
                'Regular connectivity monitoring',
                'Offline operation modes for critical systems'
            ]
        }
    ],
    'operational_risks': [
        {
            'risk': 'Occupant Comfort Issues',
            'description': 'Aggressive optimization may compromise thermal comfort',
            'probability': 'Medium',
            'impact': 'High',
            'mitigation': [
                'Conservative comfort bounds in initial deployment',
                'User feedback systems and manual overrides',
                'Gradual optimization parameter tuning',
                'Comfort monitoring and alerts'
            ]
        },
        {
            'risk': 'Equipment Wear and Reliability',
            'description': 'Frequent switching or non-standard operation may increase wear',
            'probability': 'Low',
            'impact': 'Medium',
            'mitigation': [
                'Implement equipment protection limits',
                'Monitor switching frequency and duty cycles',
                'Regular maintenance scheduling',
                'Equipment health monitoring systems'
            ]
        },
        {
            'risk': 'Energy Cost Increases',
            'description': 'Optimization errors could increase rather than decrease costs',
            'probability': 'Low',
            'impact': 'Medium',
            'mitigation': [
                'Conservative optimization limits initially',
                'Real-time cost monitoring and alerts',
                'Automatic rollback to baseline operation',
                'Regular optimization performance reviews'
            ]
        }
    ],
    'financial_risks': [
        {
            'risk': 'Lower Than Expected Savings',
            'description': 'Actual savings may be less than projected',
            'probability': 'Medium',
            'impact': 'Medium',
            'mitigation': [
                'Conservative savings estimates in business case',
                'Phased implementation to validate savings early',
                'Regular savings measurement and reporting',
                'Adjust strategies based on actual performance'
            ]
        },
        {
            'risk': 'Technology Obsolescence',
            'description': 'Implemented solutions may become outdated',
            'probability': 'Low',
            'impact': 'Medium',
            'mitigation': [
                'Choose open standards and flexible architectures',
                'Plan for regular technology updates',
                'Modular system design for easy upgrades',
                'Vendor independence where possible'
            ]
        },
        {
            'risk': 'Regulatory Changes',
            'description': 'Energy market or building regulations may change',
            'probability': 'Medium',
            'impact': 'Low',
            'mitigation': [
                'Stay informed of regulatory developments',
                'Design flexible systems adaptable to changes',
                'Participate in industry standards development',
                'Legal compliance monitoring'
            ]
        }
    ]
}

# Calculate risk scores
risk_matrix = {
    ('Low', 'Low'): 1,
    ('Low', 'Medium'): 2,
    ('Low', 'High'): 3,
    ('Medium', 'Low'): 2,
    ('Medium', 'Medium'): 4,
    ('Medium', 'High'): 6,
    ('High', 'Low'): 3,
    ('High', 'Medium'): 6,
    ('High', 'High'): 9
}

all_risks = (
    risk_assessment['technical_risks'] + 
    risk_assessment['operational_risks'] + 
    risk_assessment['financial_risks']
)

for risk in all_risks:
    risk['risk_score'] = risk_matrix.get((risk['probability'], risk['impact']), 4)

# Sort risks by score
all_risks.sort(key=lambda x: x['risk_score'], reverse=True)

print("Risk Assessment and Mitigation Strategies:")
print("=" * 80)

print(f"\nTop Risk Priorities:")
for i, risk in enumerate(all_risks[:5], 1):  # Top 5 risks
    print(f"\n{i}. {risk['risk']} (Score: {risk['risk_score']}/9)")
    print(f"   Probability: {risk['probability']}, Impact: {risk['impact']}")
    print(f"   Description: {risk['description']}")
    print(f"   Mitigation strategies:")
    for mitigation in risk['mitigation']:
        print(f"     • {mitigation}")

# Risk mitigation timeline
print(f"\n\nRisk Mitigation Implementation Timeline:")
print("=" * 60)

critical_mitigations = [
    {
        'action': 'Implement comprehensive monitoring system',
        'timeline': 'Month 1-2',
        'priority': 'Critical',
        'cost': 25000,
        'addresses': ['Model Accuracy Degradation', 'System Integration Failures', 'Energy Cost Increases']
    },
    {
        'action': 'Develop manual override and fallback systems',
        'timeline': 'Month 1-3',
        'priority': 'Critical',
        'cost': 20000,
        'addresses': ['System Integration Failures', 'Occupant Comfort Issues']
    },
    {
        'action': 'Establish conservative operational limits',
        'timeline': 'Month 1',
        'priority': 'High',
        'cost': 5000,
        'addresses': ['Occupant Comfort Issues', 'Equipment Wear and Reliability']
    },
    {
        'action': 'Deploy redundant communication systems',
        'timeline': 'Month 2-4',
        'priority': 'High',
        'cost': 30000,
        'addresses': ['Communication System Failures']
    },
    {
        'action': 'Create performance measurement framework',
        'timeline': 'Month 1-2',
        'priority': 'High',
        'cost': 15000,
        'addresses': ['Lower Than Expected Savings', 'Model Accuracy Degradation']
    }
]

print(f"{'Mitigation Action':40s} {'Timeline':12s} {'Priority':10s} {'Cost (CZK)':12s}")
print("-" * 80)

total_mitigation_cost = 0
for mitigation in critical_mitigations:
    total_mitigation_cost += mitigation['cost']
    print(f"{mitigation['action'][:39]:40s} {mitigation['timeline']:12s} "
          f"{mitigation['priority']:10s} {mitigation['cost']:11,.0f}")

print("-" * 80)
print(f"{'TOTAL RISK MITIGATION COST':40s} {'':12s} {'':10s} {total_mitigation_cost:11,.0f}")

# Calculate risk-adjusted ROI
original_portfolio_investment = optimization_economics['portfolio_metrics']['total_investment_czk']
risk_adjusted_investment = original_portfolio_investment + total_mitigation_cost
annual_savings = optimization_economics['portfolio_metrics']['total_annual_savings_czk']

# Apply risk discount to savings (assume 10% reduction due to risks)
risk_adjusted_savings = annual_savings * 0.9

risk_adjusted_payback = risk_adjusted_investment / risk_adjusted_savings
risk_adjusted_roi_5yr = (risk_adjusted_savings * 5 - risk_adjusted_investment) / risk_adjusted_investment

print(f"\n\nRisk-Adjusted Financial Analysis:")
print("=" * 50)
print(f"Original investment: {original_portfolio_investment:,.0f} CZK")
print(f"Risk mitigation cost: {total_mitigation_cost:,.0f} CZK")
print(f"Total risk-adjusted investment: {risk_adjusted_investment:,.0f} CZK")
print(f"")
print(f"Original annual savings: {annual_savings:,.0f} CZK")
print(f"Risk-adjusted annual savings: {risk_adjusted_savings:,.0f} CZK")
print(f"")
print(f"Risk-adjusted payback period: {risk_adjusted_payback:.1f} years")
print(f"Risk-adjusted 5-year ROI: {risk_adjusted_roi_5yr*100:+.1f}%")
print(f"Risk-adjusted 5-year NPV: {risk_adjusted_savings * 5 - risk_adjusted_investment:,.0f} CZK")

# Store risk analysis
risk_analysis_results = {
    'risk_assessment': risk_assessment,
    'top_risks': all_risks[:5],
    'mitigation_timeline': critical_mitigations,
    'risk_adjusted_economics': {
        'total_investment_czk': risk_adjusted_investment,
        'mitigation_cost_czk': total_mitigation_cost,
        'annual_savings_czk': risk_adjusted_savings,
        'payback_years': risk_adjusted_payback,
        'roi_5_year': risk_adjusted_roi_5yr,
        'npv_5_year_czk': risk_adjusted_savings * 5 - risk_adjusted_investment
    }
}

## 8. Summary and Final Recommendations

Generate comprehensive summary and actionable recommendations

In [ ]:
print("\nENERGY OPTIMIZATION ANALYSIS - EXECUTIVE SUMMARY")
print("=" * 80)

# Generate executive summary
executive_summary = {
    'analysis_scope': {
        'systems_analyzed': len(optimization_metrics),
        'opportunities_identified': len(optimization_opportunities),
        'total_investment_required': risk_adjusted_investment,
        'expected_annual_savings': risk_adjusted_savings,
        'payback_period': risk_adjusted_payback,
        'five_year_roi': risk_adjusted_roi_5yr * 100
    }
}

print(f"\nANALYSIS SCOPE:")
print(f"  Energy systems analyzed: {executive_summary['analysis_scope']['systems_analyzed']}")
print(f"  Optimization opportunities identified: {executive_summary['analysis_scope']['opportunities_identified']}")
print(f"")
print(f"FINANCIAL SUMMARY:")
print(f"  Total investment required: {executive_summary['analysis_scope']['total_investment_required']:,.0f} CZK")
print(f"  Expected annual savings: {executive_summary['analysis_scope']['expected_annual_savings']:,.0f} CZK")
print(f"  Payback period: {executive_summary['analysis_scope']['payback_period']:.1f} years")
print(f"  Five-year ROI: {executive_summary['analysis_scope']['five_year_roi']:+.1f}%")

# Top recommendations
top_recommendations = [
    "Implement Phase 1 quick wins immediately - lowest risk, fastest payback",
    "Deploy comprehensive monitoring system as foundation for all optimization",
    "Focus on heating system optimization - highest impact for this building type",
    "Integrate weather forecasting for predictive control strategies",
    "Establish conservative comfort bounds with manual override capabilities"
]

if system_integration.get('overall_synergy_score', 0) > 0.5:
    top_recommendations.insert(1, "Pursue integrated multi-system optimization - high synergy potential detected")

if len(quick_wins) > 0:
    top_recommendations.insert(0, f"Prioritize {len(quick_wins)} quick-win opportunities with <2 year payback")

print(f"\nTOP STRATEGIC RECOMMENDATIONS:")
for i, rec in enumerate(top_recommendations[:7], 1):
    print(f"{i}. {rec}")

# Key success factors
success_factors = [
    "Strong project management and phased implementation approach",
    "Comprehensive monitoring and measurement systems",
    "User acceptance and change management",
    "Technical expertise in energy management systems",
    "Continuous optimization and performance improvement",
    "Risk mitigation and contingency planning"
]

print(f"\nKEY SUCCESS FACTORS:")
for i, factor in enumerate(success_factors, 1):
    print(f"{i}. {factor}")

# Implementation priorities
print(f"\nIMMEDIATE ACTION ITEMS (Next 30 days):")
immediate_actions = [
    "Secure management approval and budget allocation",
    "Establish project team and assign responsibilities",
    "Begin Phase 1 quick wins implementation",
    "Deploy basic monitoring and measurement systems",
    "Develop detailed implementation plans for Phase 2"
]

for i, action in enumerate(immediate_actions, 1):
    print(f"{i}. {action}")

# Expected outcomes
print(f"\nEXPECTED OUTCOMES:")

# Calculate percentage improvements
baseline_annual_cost = baseline_metrics['total_annual_cost_czk']
savings_percentage = (risk_adjusted_savings / baseline_annual_cost) * 100

outcomes = [
    f"Energy cost reduction: {savings_percentage:.1f}% ({risk_adjusted_savings:,.0f} CZK annually)",
    f"Improved system efficiency and reduced maintenance costs",
    f"Enhanced occupant comfort through predictive control",
    f"Reduced carbon footprint and environmental impact",
    f"Improved energy security and grid independence",
    f"Foundation for future smart building technologies"
]

for i, outcome in enumerate(outcomes, 1):
    print(f"{i}. {outcome}")

# Risk considerations
print(f"\nKEY RISK CONSIDERATIONS:")
key_risks = [
    "Technology complexity may require specialized expertise",
    "Occupant comfort must be maintained during optimization",
    "System integration risks require careful testing and validation",
    "Ongoing maintenance and model updates needed for sustained performance",
    "External factors (weather, energy prices) may affect actual savings"
]

for i, risk in enumerate(key_risks, 1):
    print(f"{i}. {risk}")

# Create summary metrics visualization
if len(optimization_opportunities) > 0:
    print(f"\n\nOPTIMIZATION OPPORTUNITY MATRIX:")
    print("=" * 60)
    
    # Create impact vs effort matrix
    impact_effort_matrix = {}
    
    for opp in optimization_opportunities:
        impact = opp['impact']
        effort = opp['effort']
        key = f"{impact}-{effort}"
        
        if key not in impact_effort_matrix:
            impact_effort_matrix[key] = []
        impact_effort_matrix[key].append(opp['opportunity'])
    
    # Display matrix
    efforts = ['Low', 'Medium', 'High']
    impacts = ['High', 'Medium', 'Low']
    
    print(f"{'Impact/Effort':15s} {' Low':15s} {' Medium':15s} {' High':15s}")
    print("-" * 65)
    
    for impact in impacts:
        row = f"{impact:15s}"
        for effort in efforts:
            key = f"{impact}-{effort}"
            count = len(impact_effort_matrix.get(key, []))
            if count > 0:
                row += f" {count:2d} opps      "
            else:
                row += f" {'':10s}  "
        print(row)
    
    print("\nMatrix Guide:")
    print("  High Impact + Low Effort = Quick Wins (Prioritize)")
    print("  High Impact + High Effort = Major Projects (Plan carefully)")
    print("  Low Impact + Low Effort = Fill-in projects (When resources available)")
    print("  Low Impact + High Effort = Avoid (Unless strategic importance)")

print(f"\n" + "=" * 80)
print(f"END OF ENERGY OPTIMIZATION ANALYSIS")
print(f"=" * 80)

In [ ]:
# Save comprehensive optimization analysis results
import pickle
from pathlib import Path

# Create comprehensive results dictionary
optimization_analysis_results = {
    'analysis_metadata': {
        'analysis_date': datetime.now(),
        'systems_analyzed': list(optimization_metrics.keys()),
        'data_sources': list(analysis_results.keys())
    },
    'optimization_metrics': optimization_metrics,
    'system_integration': system_integration,
    'optimization_opportunities': optimization_opportunities,
    'optimization_economics': optimization_economics,
    'implementation_roadmap': implementation_roadmap,
    'risk_analysis': risk_analysis_results,
    'executive_summary': executive_summary,
    'recommendations': {
        'top_strategic': top_recommendations,
        'immediate_actions': immediate_actions,
        'success_factors': success_factors,
        'expected_outcomes': outcomes
    }
}

# Save to files
results_dir = Path('./data/processed')
results_dir.mkdir(parents=True, exist_ok=True)

# Save as pickle for programmatic use
with open(results_dir / 'energy_optimization_analysis.pkl', 'wb') as f:
    pickle.dump(optimization_analysis_results, f)

# Save opportunities as CSV
opportunities_df = pd.DataFrame(optimization_opportunities)
opportunities_df.to_csv(results_dir / 'optimization_opportunities.csv', index=False)

# Save cost-benefit analysis as CSV
cost_benefit_df = pd.DataFrame(cost_benefit_analysis)
cost_benefit_df.to_csv(results_dir / 'optimization_cost_benefit.csv', index=False)

# Save executive summary report
with open(results_dir / 'energy_optimization_executive_summary.txt', 'w') as f:
    f.write("ENERGY OPTIMIZATION ANALYSIS - EXECUTIVE SUMMARY\n")
    f.write("=" * 60 + "\n\n")
    
    f.write(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}\n")
    f.write(f"Systems Analyzed: {len(optimization_metrics)}\n")
    f.write(f"Opportunities Identified: {len(optimization_opportunities)}\n\n")
    
    f.write("FINANCIAL SUMMARY:\n")
    f.write(f"  Total Investment Required: {risk_adjusted_investment:,.0f} CZK\n")
    f.write(f"  Expected Annual Savings: {risk_adjusted_savings:,.0f} CZK\n")
    f.write(f"  Payback Period: {risk_adjusted_payback:.1f} years\n")
    f.write(f"  Five-Year ROI: {risk_adjusted_roi_5yr*100:+.1f}%\n")
    f.write(f"  Five-Year NPV: {risk_adjusted_savings * 5 - risk_adjusted_investment:,.0f} CZK\n\n")
    
    f.write("TOP STRATEGIC RECOMMENDATIONS:\n")
    for i, rec in enumerate(top_recommendations, 1):
        f.write(f"{i}. {rec}\n")
    
    f.write("\nIMMEDIATE ACTION ITEMS:\n")
    for i, action in enumerate(immediate_actions, 1):
        f.write(f"{i}. {action}\n")
    
    f.write("\nPHASE 1 QUICK WINS (0-6 months):\n")
    phase1_opps = phases.get('Phase 1 - Quick Wins (0-6 months)', {}).get('opportunities', [])
    for opp in phase1_opps:
        f.write(f"  • {opp['opportunity']}: {opp['implementation_cost_czk']:,.0f} CZK, "
                f"{opp['annual_savings_czk']:,.0f} CZK/year\n")
    
    f.write("\nKEY SUCCESS FACTORS:\n")
    for i, factor in enumerate(success_factors, 1):
        f.write(f"{i}. {factor}\n")
    
    f.write("\nTOP RISKS AND MITIGATIONS:\n")
    for risk in all_risks[:3]:  # Top 3 risks
        f.write(f"  Risk: {risk['risk']}\n")
        f.write(f"    Mitigation: {risk['mitigation'][0]}\n")

# Save detailed implementation plan
with open(results_dir / 'implementation_roadmap.txt', 'w') as f:
    f.write("ENERGY OPTIMIZATION IMPLEMENTATION ROADMAP\n")
    f.write("=" * 50 + "\n\n")
    
    for phase_name, phase_info in phases.items():
        if phase_info['opportunities']:
            f.write(f"{phase_name}\n")
            f.write(f"Focus: {phase_info['focus']}\n")
            f.write(f"Investment: {sum([opp['implementation_cost_czk'] for opp in phase_info['opportunities']]):,.0f} CZK\n")
            f.write(f"Annual Savings: {sum([opp['annual_savings_czk'] for opp in phase_info['opportunities']]):,.0f} CZK\n")
            f.write("\nOpportunities:\n")
            for opp in phase_info['opportunities']:
                f.write(f"  • {opp['opportunity']}\n")
                f.write(f"    Cost: {opp['implementation_cost_czk']:,.0f} CZK\n")
                f.write(f"    Savings: {opp['annual_savings_czk']:,.0f} CZK/year\n")
                f.write(f"    Payback: {opp['payback_years']:.1f} years\n\n")
            f.write("\n")

print("\nEnergy optimization analysis results saved to:")
print(f"  - {results_dir / 'energy_optimization_analysis.pkl'}")
print(f"  - {results_dir / 'optimization_opportunities.csv'}")
print(f"  - {results_dir / 'optimization_cost_benefit.csv'}")
print(f"  - {results_dir / 'energy_optimization_executive_summary.txt'}")
print(f"  - {results_dir / 'implementation_roadmap.txt'}")

print(f"\n✅ Energy optimization analysis completed successfully!")
print(f"\nKey Deliverables:")
print(f"  📊 {len(optimization_opportunities)} optimization opportunities identified")
print(f"  💰 {risk_adjusted_savings:,.0f} CZK annual savings potential")
print(f"  ⏱️ {risk_adjusted_payback:.1f} year payback period")
print(f"  📈 {risk_adjusted_roi_5yr*100:+.1f}% five-year ROI")
print(f"  🎯 {len(quick_wins)} quick wins with <2 year payback")